## FASE 2: PRÉ-PROCESSAMENTO DOS DADOS

#### Esta fase justifica as transformações feitas para tornar o modelo de Machine Learning viável.

### 2.1: Pré-processamento e Escalonamento de Dados

## Fase 2: Preparação dos Dados (Data Preparation)

### Justificativa

Após entender a estrutura e o comportamento dos dados na Fase 1 (EDA), esta fase tem como objetivo preparar o dataset para a modelagem de forma **consistente, reproduzível e livre de vazamento de dados**. Em vez de aplicar transformações “soltas” diretamente nos DataFrames, todo o pré-processamento é encapsulado em um objeto único (`preprocessor.joblib`), que será reutilizado na fase de modelagem (notebook 3) e, posteriormente, no deployment (notebook 4).

Essa abordagem segue as boas práticas de projetos de Machine Learning em produção:

- garante que as mesmas transformações sejam aplicadas em treino, validação, teste e produção;
- reduz risco de data leakage (parâmetros de imputação e escala são aprendidos apenas no conjunto de treino);
- facilita integração com pipelines de validação cruzada e tuning de hiperparâmetros.

---

### Etapas

1. **Carregamento do dataset bruto**  
   - Leitura do arquivo `creditcard.csv` diretamente do repositório do Kaggle.  
   - Verificação das dimensões iniciais do dataset.

2. **Definição formal da variável-alvo e das features**  
   - Definição de `Class` como coluna alvo (`target_col`).  
   - Identificação de todas as colunas numéricas.  
   - Remoção explícita da coluna alvo da lista de features numéricas.  
   - Criação de listas `numeric_cols` e `categorical_cols` que serão a referência oficial do projeto.

3. **Construção do pré-processador oficial (`preprocessor`)**  
   - Imputação de valores ausentes em variáveis numéricas com a mediana (`SimpleImputer(strategy="median")`).  
   - Escalonamento robusto (`RobustScaler`) para reduzir o impacto de outliers em features como `Time` e `Amount`.  
   - (Opcional) Imputação e codificação one-hot para variáveis categóricas, caso existam em versões futuras do dataset.  
   - Encapsulamento dessas transformações em um `ColumnTransformer`, compatível com `Pipeline`, `Cross-Validation` e `RandomizedSearchCV`.

4. **Persistência do pré-processador**  
   - Salvamento do objeto `preprocessor` em `artifacts/preprocessor.joblib`.  
   - Salvamento de metadados de pré-processamento (nome da coluna alvo, listas de colunas numéricas e categóricas) em `artifacts/preprocessing_metadata.json`.  
   - Esses artefatos serão carregados na Fase 3 (Modelagem) para construir os pipelines completos de treino e tuning, garantindo consistência entre notebooks e ambientes.

---

### Benefícios

- **Consistência entre as fases do projeto**: o mesmo pré-processamento é utilizado em todas as etapas (treino, validação, teste, produção), evitando discrepâncias entre notebooks e scripts.
- **Redução de vazamento de dados (data leakage)**: parâmetros de imputação e escala são aprendidos apenas nos dados de treino, quando o `preprocessor` é usado em conjunto com validação cruzada e pipelines completos.
- **Modularidade e reuso**: o objeto `preprocessor` pode ser acoplado a diferentes modelos (Logistic Regression, Random Forest, XGBoost, LightGBM, CatBoost) sem duplicação de código.
- **Facilidade de experimentação**: qualquer alteração nas regras de pré-processamento (imputação, escala, encoding) é feita em um único ponto (`pipeline_new.py`), refletindo automaticamente em todas as fases do projeto.
- **Preparação para produção**: o uso de artefatos versionáveis (`joblib` + `metadata.json`) aproxima o notebook de um fluxo real de MLOps, facilitando o deploy e o monitoramento do modelo final.


### Célula 1 – Imports e ligação com pipeline_new.py

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import joblib # Para salvar os objetos transformados

project_root = r"C:\Users\debor\OneDrive\Github\FraudSense"

if project_root not in sys.path:
    sys.path.append(project_root)

from pipeline_new import build_preprocessor, save_pipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
...


Ellipsis

### Célula 2 – Carregamento dos dados com load_data()

In [8]:
# Célula 2 Carregamento dos Dados
def load_data():
    """Carrega o dataset original"""
    # Se você já baixou no notebook 1, o arquivo deve estar em cache ou na pasta local
    # Ajuste o caminho se necessário, ou use o kagglehub novamente
    import kagglehub
    path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
    file_path = os.path.join(path, 'creditcard.csv')
    return pd.read_csv(file_path)

df = load_data()
print(f"Dados carregados: {df.shape}")

Dados carregados: (284807, 31)


### Célula 3 – Definição de colunas oficiais

In [9]:
# Célula 3: definição de colunas para o pipeline oficial

# Target
target_col = "Class"

# Features numéricas (no dataset creditcard.csv, praticamente todas são numéricas)
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# Remover a coluna alvo da lista de numéricas
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

# Se você tiver colunas categóricas em algum momento futuro, coloque aqui
categorical_cols = []  # no creditcard.csv original não há categorias

print("Colunas numéricas usadas no pipeline:")
print(numeric_cols[:10], "... (total:", len(numeric_cols), ")")
print("Colunas categóricas usadas no pipeline:")
print(categorical_cols)


Colunas numéricas usadas no pipeline:
['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9'] ... (total: 30 )
Colunas categóricas usadas no pipeline:
[]


### Célula 4 – Construção do pré-processador oficial

In [10]:
# Célula 4: construção do pré-processador oficial para o projeto

from pipeline_new import build_preprocessor

preprocessor = build_preprocessor(
    numeric_cols=numeric_cols,
    categorical_cols=categorical_cols,
    numeric_strategy="median",  # igual ao que você já vinha usando
    scale="robust"              # coerente com seu uso de RobustScaler
)

print("Pré-processador criado com sucesso!")
print(type(preprocessor))
print("Transformers definidos:")
for name, trans, cols in preprocessor.transformers:
    print(f" • {name}: {type(trans).__name__} -> {len(cols)} colunas")


Pré-processador criado com sucesso!
<class 'sklearn.compose._column_transformer.ColumnTransformer'>
Transformers definidos:
 • num: Pipeline -> 30 colunas


### Célula 5 – Salvando o pré-processador oficial

In [11]:
# Célula 5: salvar o pré-processador oficial

from pipeline_new import save_pipeline

os.makedirs("artifacts", exist_ok=True)

preprocessor_path = os.path.join("artifacts", "preprocessor.joblib")
save_pipeline(preprocessor, preprocessor_path)

print("\nPré-processador salvo com sucesso!")
print("Caminho:", preprocessor_path)
print("Este é o objeto que será usado na fase de modelagem (notebook 3).")


Pipeline salvo em artifacts\preprocessor.joblib

Pré-processador salvo com sucesso!
Caminho: artifacts\preprocessor.joblib
Este é o objeto que será usado na fase de modelagem (notebook 3).
